# HW1 Q1.1 [30 pts]



## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> add any cells to this Jupyter Notebook, because that will crash the autograder.
</div>


All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  



<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>


`get_colors()`

`user()`

`get_sets()`

`get_top_parts()`

`build_graph()`

`in_degree_for_node()`

`out_degree_for_node()`

`max_in_degree()`

`average_in_degree()`

`filter_graph_min_in_degree()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from Rebrickable during auto-grading. You do not need to write code for unreasonable scenarios, e.g., handling a non-existent node id, or an invalid quantity specification.  

We will import and auto-grade the two cells containing the `Graph` class and the `get_colors()` function since they contain all of the required implementations.  We will **NOT** grade the other cells.    

Since the overall correctness of your code may require multiple methods to work together correctly (i.e., some methods are interdepedent), implementing only a subset of the methods likely will lead to a low score.

You **MUST** complete Q1.1 before Q1.2, since the output files from this notebook are used in Q1.2.  

### Helper functions

You are permitted to write additional helper functions or methods, or use additional instance variables within the `Graph` class so long as the previously described functions work as required.  

<div class="alert alert-block alert-danger">
    Do <strong>NOT</strong> remove or modify the following utility functions:
</div>


`show_graph_info()`

`write_nodes_file()`

`write_edges_file()`

`write_adjacency_list()`

However, you should verify that they work properly with the rest of your `Graph` class implementation. _i.e_., ensure that calling `write_edges_file('graph.csv')` is writing out a _.csv_ representation of your graph.  We will call these functions during auto-grading to write out your graph files.  

## Rebrickable Familiarization - Acquiring Domain Knowledge
#### Watch the introduction [video](https://youtu.be/t1DtZyyVJvQ). 

## a. [2 pts] Warmup - Download data using the Rebrickable API

First, reference the example here on use of `http.client` 

https://docs.python.org/3/library/http.client.html#examples

This section is important for you to gain an understanding of how to get data from Rebrickable.  In the following sections, you will download information about Lego Sets and Parts to build construct a graph.  

As a warm-up, complete the method to download a list of part colors from the Rebrickable API.  In the API docs, find the first API call under 'lego'.  Experiment with the 'Try it out!' feature to get a sense of what data it will return.  

The function must accept the following arguments:

`quantity`:  a positive integer that limits the amount of returned results. _e.g_., the call `get_colors(quantity=5, api_key = '999asdf')` would return a list of 5 colors.  

`api_key`:  a string that accepts a Rebrickable API key.  Use your api key that we instructed you to get in Q1.1 of the HW1 document.  After you implement this function, you may delete your api key from this Jupyter notebook. We will use our own key for grading.  



In [1]:
import http.client
import json
import time
import timeit
import pickle

def get_colors(quantity=5, api_key=None):
    conn = http.client.HTTPSConnection("rebrickable.com")
    key = api_key
    auth_token = {'Authorization': 'key '+key}
    payload = "{}"
    headers = auth_token
    params = '/api/v3/lego/colors/?page=1&page_size=' + str(quantity) # modify this so that results are limited by the `quantity` argument.

    conn.request("GET", params, payload, headers=headers)
    
    ################################################################
    #  insert code to handle data returned in the response         #
    # return a list of strings, one string for each color returned #        
    ################################################################  
    
    response = conn.getresponse()
    data = json.load(response)['results']
    
    colors = []
    for s in data:
        colors.append(s['name'])
    
    return colors

# uncomment these next 2 lines to test your implementation
colors = get_colors(quantity=5, api_key='0f0c490e4ae5dc9b116f0009dcb4f5f8')
print(colors)

# The following is sample output of the get_colors() function and is used only to exmplify 
# the format of the results for this cell. 
# you may comment out or delete the below line.

# print("\n-------------sample-output-------------")
# print(['Fuschia', 'Magenta', 'Violet', 'Pink', 'White'])


['Black', 'Blue', 'Green', 'Dark Turquoise', 'Red']


### Class `Graph`

You will implement the class step-by-step in the instructions contained **below for parts b-d**.  Note that you will need to re-run this cell after you make changes to your code in class `Graph` in order for the subsequent cells below to run properly.  If this is your first time running this notebook, make sure to run this cell before continuing so that the examples below will work.  

If you need to add additional modules for your assignment, add them at the beginning of this cell.  You may only use modules and libraries from the [Python Standard Library](https://docs.python.org/3/library/).  

In [2]:
import http.client
import json
import time
import timeit
import pickle

class Graph:

    def __init__(self, with_file=None): 

        if with_file is None:
            self.adjacency_list = {}
        else:   
            with open(with_file, 'rb') as handle:
                self.adjacency_list = pickle.load(handle)
                
        self.api_key = None
    
    @staticmethod
    def user():
        """
        :return: string
        your GTUsername, NOT your 9-Digit GTId  
        """         
        return 'rliang37'

    def show_graph_info(self):
        """
        :return: none
        prints out the contents of the adjacency list 
        """
        num_nodes = len(self.adjacency_list)
        print("nodes: " + str(num_nodes))
        
        num_edges = 0
        for n in self.adjacency_list.keys():
            num_edges += len(self.adjacency_list[n])
        print("edges: " + str(num_edges))
        
        print("------------------ADJACENCY--LIST---------------------")
        for n in self.adjacency_list.keys():
            print(str(n) + ": " + str(self.adjacency_list[n]))


    def get_sets(self, quantity=5, order_by=None):
        
        """
        :param quantity: integer 
        :param order_by: string
        :return: list of strings, each string is a set_num representing a set.   
        """
        
        if not isinstance(order_by, str):            
            raise ValueError("order_by argument must be a string")        

        lego_sets = []
        
        ################################################################
        #    code to download and process lego sets data               #
        #                                                              #        
        ################################################################
        conn = http.client.HTTPSConnection("rebrickable.com")
        key = self.api_key
        auth_token = {'Authorization': 'key '+key}
        payload = "{}"
        headers = auth_token
        params = '/api/v3/lego/sets/?page=1&page_size=' + str(quantity)
        if order_by != None:
            params += '&ordering=' + order_by

        conn.request("GET", params, payload, headers=headers)  
    
        response = conn.getresponse()
        data = json.load(response)['results']
    
        for s in data:
            lego_sets.append(s['set_num'])
    
        return lego_sets

    
    def get_top_parts(self, set_num, max_top_parts=5):
        
        """
        :param set_num: string, a valid set_num
        :param max_top_parts: integer, the number of parts to retrieve for the set_num  
        :return: list of strings, each string is a part_num used by the set. 
        """
        
        if not isinstance(set_num, str):            
            raise ValueError("set_num must be a string")        
            
        top_parts = []
        unsorted_top_parts = []
    
        #################################################################
        #       code to download and process lego parts data            #
        #                                                               #        
        #################################################################
        conn = http.client.HTTPSConnection("rebrickable.com")
        key = self.api_key
        auth_token = {'Authorization': 'key '+key}
        payload = "{}"
        headers = auth_token
        params = '/api/v3/lego/sets/' + set_num + '/parts/' + '?pagesize=1000'

        conn.request("GET", params, payload, headers=headers)  
    
        response = conn.getresponse()
        data = response.read().decode()
        json_response = json.loads(data)
        result = json_response['results']
        next_page = json_response['next']
        
        while next_page != None:
            next_page = next_page[23:]
            conn.request("GET", next_page, payload, headers=headers)
            response = conn.getresponse()
            data = response.read().decode()
            json_res = json.loads(data)
            result.extend(json_res["results"])
            next_page = json_res["next"]
            conn.close()
        
        dict = {}
        
        for s in result:
            if s["is_spare"] != True:
                q = dict.get((s["part"])["part_num"], 0) 
                dict[(s["part"])["part_num"]] = s["quantity"] + q
                
        ans = []
        ans = sorted(dict.items(), key = lambda x:x[1], reverse = True)
        
        if max_top_parts > len(ans):
            max_top_parts = len(ans)
            
        for i in range(max_top_parts):
            top_parts.append(ans[i][0])
            
        return top_parts


    def build_graph(self, sets_quantity=5, top_parts_quantity=5, set_ordering=None):
        
        """
        Downloads data and builds an adjacency list representation of a graph.
        :param sets_quantity: integer, passed to the get_sets() `quantity` argument
        :param top_parts_quantity: integer, passed to the get_top_parts() 'max_top_parts' argument. 
        :param set_ordering: string, passed to the get_sets() 'order_by' argument.
        :return: None
        """
        
        set_nums = self.get_sets(quantity=sets_quantity, order_by=set_ordering)
        
        for set_num in set_nums:  
            
            #################################################################
            #       code to add Lego Set as a node to the adjacency list    #
            #                                                               #        
            #################################################################  
            if set_num not in self.adjacency_list.keys():
                self.adjacency_list[set_num] = set()
            
            top_parts = self.get_top_parts(set_num, max_top_parts=top_parts_quantity)

            for part_num in top_parts:

                #################################################################
                #       code to add Lego as a node to the adjacency list        #
                #       then, add edges from lego sets to parts                 #        
                #################################################################  
                if part_num not in self.adjacency_list.keys():
                    self.adjacency_list[part_num] = set()
                self.adjacency_list[set_num].add(part_num)
                


    def out_degree_for_node(self, node_id):
        """
        Calculates the out-degree for a valid node id present in the adjacency list (graph)
        :param node_id: string
        :return: integer, the calculated out-degree
        """
        if not isinstance(node_id, str):            
            raise ValueError("node_id must be a string")
        if node_id not in self.adjacency_list.keys():
            raise KeyError("node_id not found in adjacency list")
        
        #################################################################
        #       code to calculate and return out-degree.                #
        #                                                               #        
        #################################################################   
        return len(self.adjacency_list[node_id])
    

    def in_degree_for_node(self, node_id):
        """
        Calculates the in-degree for a valid node id present in the adjacency list (graph)
        :param node_id: string
        :return: integer, the calculated in-degree
        """
        if not isinstance(node_id, str):            
            raise ValueError("node_id must be a string")
        if node_id not in self.adjacency_list.keys():
            raise KeyError("node_id not found in adjacency list")
        
        #################################################################
        #       code to calculate and return in-degree.                 #
        #                                                               #        
        #################################################################
        count = 0
        for source_node in self.adjacency_list.keys():
            if node_id in self.adjacency_list[source_node]:
                count += 1
        return count
   
    
    def max_in_degree(self):
        
        """ 
        Find the node id and in-degree for the node having the highest in degree
        :return: tuple, (string, integer).  The string is the node id, the integer is the in-degree
          
        """

        #################################################################
        #       code to calculate and return max in-degree.             #
        #                                                               #        
        #################################################################
        max_node_id = ""
        max_in_degree = 0
        
        for node_id in self.adjacency_list.keys():
            in_degree = self.in_degree_for_node(node_id)
            if in_degree> max_in_degree:
                max_node_id = node_id
                max_in_degree = in_degree
    
        return (max_node_id, max_in_degree)
    
    def average_in_degree(self):
        """
        Calculate the average in-degree of all nodes having an in_degree > 0.    
        :return: decimal
        """
        #################################################################
        #       code to calculate and return avg in-degree.             #
        #                                                               #        
        #################################################################
        sum_in_degree = 0
        count = 0
        
        for node_id in self.adjacency_list.keys():
            in_degree = self.in_degree_for_node(node_id)
            if in_degree > 0:
                count += 1
                sum_in_degree += in_degree
        
        return sum_in_degree / count


    def filter_graph_min_in_degree(self, in_degree=0):   
        """  
        Removes nodes in the adjacency list having an in-degree > 0 AND having an in-degree < in_degree param.
        Does not remove nodes with an in-degree = 0 
        :param in_degree: integer 
        :return: None
        """
        
        #################################################################
        #                      code to filter graph                     #
        #                                                               #        
        #################################################################  
        for node_id in list(self.adjacency_list.keys()):
            node_in_degree = self.in_degree_for_node(node_id)
            if node_in_degree > 0 and node_in_degree < in_degree:
                del self.adjacency_list[node_id]
                for k in self.adjacency_list.keys():
                    if node_id in self.adjacency_list[k]:
                        self.adjacency_list[k].remove(node_id)


    def write_edges_file(self,  path="graph.csv"):
        """
        write all edges out as .csv
        :param path: string
        :return: None
        """
        edges_path = path
        edges_file = open(edges_path, 'w')

        edges_file.write("source"+","+"target"+"\n")
        
        for n in self.adjacency_list.keys():
            adjacent_nodes = self.adjacency_list[n]
            for an in adjacent_nodes:
                edges_file.write(str(n) + "," + str(an) + "\n")
        
        edges_file.close()
        print("finished writing edges as "+ path)


    def write_adjacency_list(self, path="graph.pickle"):  
        """
        Serialize and write out adjacency list object as a .pickle 
        :param path: string
        :return: None
        """
        with open(path, 'wb') as handle:
            pickle.dump(self.adjacency_list, handle)
        print("finished writing adjacency list as "+ path)  

### Explore the Adjacency List data structure 

The Graph will be represented internally within the class in a data structure known as an adjaceny list.  

Before you start implementation, load the sample serialized adjacency list from the hw1-skeleton and explore what you will be building.  The next block initializes a `Graph` object with a serialized adjacency list.  The intent is to show the correct format of the adjacency list and to become familiar with its structure.  You will construct your own adjacency list to represent a graph of the data that you download from Rebrickable. 

You do not need to write any code for the next cell to work! Simply run the cell and explore the data structure.

In [3]:
graph = Graph(with_file="sample_graph.pickle")
# graph = Graph(with_file="graph.pickle")
graph.show_graph_info()

nodes: 19
edges: 21
------------------ADJACENCY--LIST---------------------
75192-1: {'3021', '15712', '2780', '6558', '3023'}
2780: set()
3023: set()
6558: set()
15712: set()
3021: set()
71043-1: {'3005', '15573', '2420', '2412b', '54200'}
54200: set()
15573: set()
3005: set()
2412b: set()
2420: set()
10256-1: {'3062b', '3024', '3005', '2877'}
3024: set()
3062b: set()
2877: set()
10189-1: {'3062b', '3024', '3005', '2877'}
SWMP-1: {'3024', '3710', '3023'}
3710: set()


The adjacency list data structure is a dictionary where each key is a string and the value is a set.  There exists one key for each node in the graph.  Watch Chris Pryby's comparison of directed and un-directed adjacency lists [here](https://youtu.be/_nHAa3j8xTE?t=496)

This is a directed graph: 
- Each Lego Set is a source node with an out-degree > 0 and an in-degree = 0
- Each Lego Part is a sink (target) with an in-degree > 0 and an out-degree = 0

## b. [17 pts] Building and producing a graph 
To build the graph, you will implement 3 functions: 
- `get_sets()`
- `get_top_parts()`
- `build_graph()`

The first two methods, `get_sets()` and `get_top_parts()` will be used within the `build_graph()` method.  During grading, we will supply different arguments to the method calls than used within the assignment to verify proper function and to discourage hard-coding of results.   

At the end of this part, you will save your graph as a _.pickle_ file and as a _.csv_ edges file using the provided utility functions.  

### i. [1 pt] Ensure all API calls use the `self.api_key` instance variable in the `Graph` class.

<div class="alert alert-block alert-info">
    <b>Note:</b> During auto-grading we will replace your api key with our own.  
    <br />
</div>



For this reason, your class **must** use the `self.api_key` instance variable _anytime_ you need to download data. _e.g_., you will likely need to refer this instance variable for `get_sets()`  or `get_top_parts()`.    
During grading, we will call your code in a manner similar to: 

In [4]:
graph = Graph()
graph.api_key = '0f0c490e4ae5dc9b116f0009dcb4f5f8'
sets = graph.get_sets(5,  "-num_parts")
print(sets)
# etc...

['BIGBOX-1', '75192-1', '71043-1', '10256-1', '10189-1']


After you complete the assignment, you may delete all instances of your api key from this Jupyter notebook.  Remember that we will only import and run your `Graph` class.   

### ii. [5 pts] Get Lego sets from the Rebrickable API in the `get_sets()` method.  

In the Rebrickable API documentation, locate the call that will **get a list of sets.**

Implement the method `get_sets()` above in the `Graph` class to download the Lego Set numbers.
The method signature **must** accept the arguments `quantity` and `order_by` where:
- `quantity` is a positive integer that determines the number of sets downloaded
- `order_by` is a string specifying how the API should sort the returned data.  

You will need to specify how to sort the data using the `order_by` argument so that the Sets are organized by the number of parts in descending order.  

Within the returned data, note that a Lego Set Number is given by the string `set_num` . The `get_sets()` method **must** return a list of `set_num`[s] of length specified by `quantity`.

In [5]:
graph = Graph()
graph.api_key = '0f0c490e4ae5dc9b116f0009dcb4f5f8' # replace with your api key  
lego_sets = graph.get_sets(quantity=5, order_by="-num_parts")
print(lego_sets)

# The following is sample output and is used only to exemplify the format of the results for this cell, 
# you may comment out or delete the the line below.

# print("\n---------sample-output-from-lego-sets----------")
# print(['20112-3', '9994-2', 'ST03-2', '4423-01', '55907'])

['BIGBOX-1', '75192-1', '71043-1', '10256-1', '10189-1']


### iii. [5 pts] Get the Lego Parts with the highest quantities in a given Lego Set inventory.  

In the Rebrickable API documentation, locate the call that will **get a list of all Inventory Parts for a Set**
 
Implement the method `get_top_parts()` in the `Graph` class to download this data.  Each part returned will contain a `quantity` value specifying how many parts are present in the set.  You **must** filter and sort this data to meet the following requirements:

1. Discard any part that has a value of `'is_spare' : true`. (This removes duplicate `part_num`s)    
2. Sort the data to return the parts with the highest quantity value.

    
e.g., Here are some sample parts data for a Lego Set that has the following 3 parts. 

{'part_num': '222', 'quantity': **25**, 'is_spare': false},     

{'part_num': '333', 'quantity': **22**, 'is_spare': false},

{'part_num': '444', 'quantity': **21**, 'is_spare': false}  

Suppose that we have passed `max_top_parts` = 2.  In this case, return the 2 parts, i.e., part_num '222' and '333' becuase they have the highest quantities for the set.   


The method signature **must** accept the arguments `set_num` and `max_top_parts` where:
- `set_num` is a string representing a valid `set_num` in the Rebrickable database. 
- `max_top_parts` is a positive integer specifying how many of the top parts must be retrieved for the `set_num`

Within the returned data, note that a Lego Part Number is given by the string `part_num`. The `get_top_parts()` **must** returns a list of `part_num`[s] of length specified by `max_top_parts`.

In [6]:
# this will take several seconds to complete.
graph = Graph()
graph.api_key = '0f0c490e4ae5dc9b116f0009dcb4f5f8'  # replace with your api_key
graph.get_sets(quantity=2, order_by="-num_parts")
data = graph.get_top_parts(set_num='71043-1', max_top_parts=5)
print(data)

# The following is sample output and is used only to exmplify the format of the results for this block, 
# you may comment out or delete the the line below.
# print("\n-------------sample-output-------------")
# print(['5600a', '7700c'])


['3024', '3023', '54200', '6141', '3005']


### iv. [5 pts] Build a directed graph relating Lego Sets and Lego Parts by implementing the `build_graph()` method. 

You will modify the `self.adjacency_list` instance variable by adding nodes and edges.  You must use the methods `get_sets()` and `get_top_parts()` within this method to retrieve the graph data. 

This is a directed graph: you will only represent the directed edge from the Lego Set as a source and the Lego Part as the target. The sample adjacency list you loaded earlier from "sample_graph.pickle" was an example of a directed graph representation.  
 

The following steps demonstrate an example of how you might construct your graph in the `build_graph()` method.

**FUNCTION** build graph 
> 
> Lego Sets = **CALL** get sets 
> 
> **FOR EACH** Lego Set in Lego Sets:
> > 
> > **IF** the Lego Set is not in the adjacency list **THEN**:
> > 
> > > add the Lego Set as a node 
> > >
> > **ENDIF**
> > 
> > Lego Parts = **CALL** get top parts
> > 
> > **FOR EACH** Lego Part in Lego Parts:	
> > 
> > > **IF** the Lego Part is not in the adjacency list **THEN**:
> > >
> > > > add the Lego Part as a node
> > > > 
> > > **ENDIF**
> > > 
> > > add edge from the Lego Set to the Lego Part
> >
> > **ENDFOR** 
> 
> **ENDFOR**
>
**ENDFUNCTION**

### v. [1 pt] Serialize and write your graph out as a `.pickle` and as a `.csv`.  

Use the **provided** utility methods in the `Graph` class to accomplish this.  You can build your graph analysis methods by loading the _.pickle_ file to avoid having to re-download the data from Rebrickable each time.  

<div class="alert alert-block alert-info">
    <b><i>graph.csv</i> will be used with Argo-Lite in Q1.2.</b>
    <br />
</div>

Build a graph with **20** sets and with the top **5** parts for each set.

In [7]:
# this will take some time to run since it is downloading and building the entire graph.  
graph = Graph()
graph.api_key = '0f0c490e4ae5dc9b116f0009dcb4f5f8'  # replace with your api_key
graph.build_graph(sets_quantity=20, top_parts_quantity=5, set_ordering="-num_parts")
graph.write_adjacency_list("graph.pickle")
graph.write_edges_file("graph.csv")

finished writing adjacency list as graph.pickle
finished writing edges as graph.csv


## c. [10 pts] Graph Exploration and Analytics

Continue implementing the `Graph` class by completing these analytics methods.  

 <div class="alert alert-block alert-info">
    <b>Note:</b> In each cell, a sample graph is loaded and a sample call is provided.  You may modify these calls as needed to test your code.  <i>e.g.</i>, replace 'my_graph.pickle' with <b>your own</b> 'graph.pickle', or replace the node id with a node id from your own graph.  
</div>


### i. [1 pt] Calculate the out-degree for a node
Implement the `out_degree_for_node()` method.  Remember that all Lego Set nodes will have an out-degree > 0.  All Lego Parts nodes will have an out-degree = 0.

In [8]:
# graph = Graph(with_file="sample_graph.pickle")
graph = Graph(with_file="graph.pickle")
graph.show_graph_info()
print(graph.out_degree_for_node("75192-1"))
print(graph.out_degree_for_node("3005"))


nodes: 56
edges: 98
------------------ADJACENCY--LIST---------------------
BIGBOX-1: {'3020', '98138', '2780', '3023', '6141'}
3023: set()
2780: set()
6141: set()
3020: set()
98138: set()
75192-1: {'3021', '15712', '2780', '3023', '2412b'}
2412b: set()
3021: set()
15712: set()
71043-1: {'3005', '3024', '3023', '6141', '54200'}
3024: set()
54200: set()
3005: set()
10256-1: {'3005', '3024', '3023', '3062b', '2877'}
3062b: set()
2877: set()
10189-1: {'3005', '3024', '3023', '3062b', '2877'}
SWMP-1: {'3020', '3024', '3023', '3068b', '3710'}
3710: set()
3068b: set()
2000409-1: {'3010', '3005pr0003', '4286', '3957b', '3004'}
3010: set()
3004: set()
3005pr0003: set()
3957b: set()
4286: set()
10179-1: {'3021', '2780', '3023', '3068b', '3710'}
70620-1: {'3069b', '3024', '3023', '3622', '3004'}
3069b: set()
3622: set()
75252-1: {'15573', '3069b', '6558', '3023', '2412b'}
6558: set()
15573: set()
75827-1: {'3005', '3010', '3069b', '3023', '3004'}
40179-1: {'3024', '96874', '4186'}
4186: set()
968

### ii. [2 pts] Calculate the in-degree for a node
Implement the `in_degree_for_node()` method.  Remember that all Lego Parts will have an in-degree >0.  All Lego Sets will have an in-degree = 0.

In [9]:
graph = Graph(with_file="sample_graph.pickle")
# graph = Graph(with_file="graph.pickle")
print(graph.in_degree_for_node("3005"))
print(graph.in_degree_for_node("75192-1"))

3
0


### iii. [1 pt] Find and return information about the node with the highest in-degree

Implement the `max_in_degree()` method.

This method must return a tuple containing a `(string, integer)` pair representing the node id and the in-degree of that node.  _e.g_., `('0970', 5)` indicates that a Lego Part with part num / node id `0970` contains the highest in-degree of 5.  

In [10]:
graph = Graph(with_file="sample_graph.pickle")
#graph = Graph(with_file="graph.pickle")
print("node with highest in-degree: " + str(graph.max_in_degree()))

node with highest in-degree: ('3005', 3)


### iv. [2 pts] Calculate the average in-degree 

Implement the `average_in_degree()` method.

In [11]:
# graph = Graph(with_file="sample_graph.pickle")
graph = Graph(with_file="graph.pickle")
print("average in-degree: " + str(graph.average_in_degree()))

average in-degree: 2.7222222222222223


**Note:** You are not required to calculate average out-degree since we have artificially limited the out-degree of each set by specifying the `top-parts` that we downloaded for it. You may implement this method to test your adjacency list.  _e.g_., no Lego Set node should have an out-degree > top_parts_quantity value passed to `get_top_parts()`

### v. [4 pts] Graph Filtering
Filter out leaf nodes, i.e., nodes with an in-degree = 1.  Filtering the nodes is accomplished by deleting the node id from the adjacency list.  You should not be filtering out any nodes with an in-degree = 0. Please feel free to discuss the reason why in Piazza. 

To check your work, consider using some of the graph analytics methods you have implemented to check your graph. For example, consider that the average in-degree of your filtered graph would not be < 2.  

In [12]:
graph = Graph(with_file="sample_graph.pickle")
graph.filter_graph_min_in_degree(2)
graph.show_graph_info()

nodes: 10
edges: 12
------------------ADJACENCY--LIST---------------------
75192-1: {'3023'}
3023: set()
71043-1: {'3005'}
3005: set()
10256-1: {'3062b', '3024', '3005', '2877'}
3024: set()
3062b: set()
2877: set()
10189-1: {'3062b', '3024', '3005', '2877'}
SWMP-1: {'3024', '3023'}


## d. [1 pt] Implement the user method

In [13]:
print(Graph.user())

rliang37
